In [ ]:
import sqlite3
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

# Initialize OpenAI client
# Ensure you have OPENAI_API_KEY set in your environment variables

DB_PATH = r'E:\SynologyDrive\ironhack\week_14\day_2\nlp-business-case-automated-customers-reviews\data\raw\gamesDB.db'


def get_reviews_by_game(db_path):
    """
    Reads reviews from the SQLite database and groups them by AppID.
    """
    if not os.path.exists(db_path):
        print(f"Error: Database file '{db_path}' not found.")
        return None

    try:
        conn = sqlite3.connect(db_path)
        # Select relevant columns.
        # distinct review text to avoid duplicates if any
        query = "SELECT appid, review, voted_up FROM reviews"
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"Error reading database: {e}")
        return None


def generate_summary(appid, game_reviews):
    """
    Generates a summary for a specific game using OpenAI API.
    """

    # Sampling to avoid token limits (e.g., take first 50 of each or random sample)
    # Adjust limit based on your token budget and model context window
    SAMPLE_LIMIT = 50
    # 1. Filter for positive reviews, sample randomly, then convert to list
    pos_reviews = game_reviews[game_reviews['voted_up'] == True].sample(
        n=min(len(game_reviews[game_reviews['voted_up'] == True]), SAMPLE_LIMIT)
    )['review'].tolist()

    # 2. Filter for negative reviews, sample randomly, then convert to list
    neg_reviews = game_reviews[game_reviews['voted_up'] == False].sample(
        n=min(len(game_reviews[game_reviews['voted_up'] == False]), SAMPLE_LIMIT)
    )['review'].tolist()

    pos_sample = pos_reviews[:SAMPLE_LIMIT]
    neg_sample = neg_reviews[:SAMPLE_LIMIT]

    pos_text = "\n- ".join([r for r in pos_sample if r and len(str(r)) > 100]) # Filter very short/empty
    neg_text = "\n- ".join([r for r in neg_sample if r and len(str(r)) > 100])
    
    print(f'text length to send to openai: len(pos_text), len(neg_text)')

    system_prompt = "You are an expert video game journalist and data analyst specializing in horror/survival horror games.\
                    Your main targets are gamer community, other game journalists and casual gamers.\
                    Use appropiate output tone for the audience avoiding informal tone but always being professional."

    user_prompt = f"""
    Analyze the following user reviews for the game (AppID: {appid}).
    
    ### Positive Reviews Sample:
    {pos_text[:8000]} # Truncate to ensure we stay within reasonable limits if reviews are long
    
    ### Negative Reviews Sample:
    {neg_text[:8000]}
    
    Based on these reviews, generate a 500 words report containing:
    1. **Top 3 Highlights**: The key reasons players recommend this game.
    2. **Critical Pain Points**: The most recurring complaints or technical issues.
    3. **The Verdict**: A final summary and witty sentence stating if the game is a "Must-Play", "Give it a try", "Hidden Gem", "Skip", "Avoid at all cost".

    CONSTRAINT: Make smooth output text. Do not hardcode every section esplicitly. It must be natural text to post it on a blog.
    """

    try:
        client = OpenAI()
        response = client.chat.completions.create(
            model="gpt-3.5-turbo", # Or gpt-4 if available/preferred
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7,
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating summary: {e}"


def main():
    print("Loading reviews from database...")
    df = get_reviews_by_game(DB_PATH)

    if df is not None and not df.empty:
        unique_games = df['appid'].unique()
        review_unique_games = df['appid'].sample(n=3)
        print(f"Found {len(unique_games)} unique games.")

        # For demonstration, we iterate through the first 3 games.
        # Remove [:3] to process all.
        for appid in review_unique_games[:3]:
            print(f"Processing Game AppID: {appid}...")
            game_reviews = df[df['appid'] == appid]

            # Skip if too few reviews to meaningful summary
            if len(game_reviews) < 5:
                print(f"Skipping {appid} (not enough reviews)")
                continue

            summary = generate_summary(appid, game_reviews)
            print("-" * 40)
            print(f"SUMMARY FOR APPID {appid}")
            print("-" * 40)
            print(summary)
            print("\n")
    else:
        print("No reviews found or could not load database.")

if __name__ == "__main__":
    main()
